#  DB초기화 프로그램 - PowerBI용 데이터 생성

In [1]:
!pip install pymysql > /dev/null #mysql 설치

# mysql.json 파일 업로드
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

# json 파일 읽기, mysql 로그인 및 접속
import json, pymysql
with open(filename) as fp:
  config_str = fp.read()
config = json.loads(config_str)

conn = pymysql.connect(**config)

Saving mysql.json to mysql.json


## product 테이블 생성

In [2]:
sql_ct_products = """
    create table if not exists products (
        pid int not null auto_increment primary key,
        pname varchar(20) not null,
        pprice int not null,
        pcategory varchar(40) not null,
        pcost int not null
        
        )auto_increment=10001;
    """
cur = conn.cursor()
cur.execute(sql_ct_products)

0

In [3]:
p_bottom_names = ['청바지', '흑청바지', '중청바지', '면바지', '스커트']
p_top_names = ['반팔티', '긴팔티', '후드티', '맨투맨', '셔츠']
p_shoes_names = ['운동화', '러닝화', '구두', '부츠', '슬리퍼']

In [4]:
def add_pds(pname, pcategory):
  cur = conn.cursor()
  import random
  import pandas as pd
  import numpy as np
  pprice = random.randint(30000, 100000)
  pcost = pprice * ((random.randint(20, 40)) / 100)

  sql_insert = "insert into products(pname, pprice, pcategory, pcost) values(%s, %s, %s, %s)"

  cur.execute(sql_insert, (pname, pprice, pcategory, pcost))
  conn.commit()
  return

In [5]:
for i in range(1, 3):
  for bt in p_bottom_names:
    add_pds(f'{bt}{i}', 'bottom')
  for tp in p_top_names:
    add_pds(f'{tp}{i}', 'top')
  for ss in p_shoes_names:
    add_pds(f'{ss}{i}', 'shoes')


## sales 테이블 생성

In [6]:
sql_ct_sales = """
    create table if not exists sales (
        sid int(20) not null auto_increment primary key,
        sdate date not null,
        scompany varchar(40) not null,
        spid int not null,
        sunit int not null,
        foreign key(spid) references products(pid)
        )auto_increment=00101;
    """
cur = conn.cursor()
cur.execute(sql_ct_sales)

0

In [7]:
def add_sales(sdate, scompany):
  import random
  import pandas as pd
  import numpy as np
  
  cur = conn.cursor()
  spid = random.randint(10001, 10030)
  sunit = random.randint(5, 10)

  sql_insert = "insert into sales(sdate, scompany, spid, sunit) values(%s, %s, %s, %s)"

  cur.execute(sql_insert, (sdate, scompany, spid, sunit))
  conn.commit()
  return

In [8]:
companys = ['무신사', 'abc마트', '이마트', '롯데백화점', '현대백화점']
import random
for year in range(2015, 2021):
  for i in range(1, 13):
    x = random.randint(2, 3)
    for k in range(x):
      for company in companys:
        sdate = f'{year}-{i}-{random.randint(1, 28)}'
        add_sales(sdate, company)


## 두 테이블을 join 한 view 생성

In [9]:
sql = """
    create view sales_list as
      select sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost, 
      pprice*sunit as revenue, (pprice-pcost)*sunit as profit
      from sales
      join products
      on sales.spid = products.pid
      order by sdate;
"""
cur = conn.cursor()
cur.execute(sql)

0

In [ ]:
# 데이터 sales_list.csv로 저장

sql = 'select * from sales_list'
cur.execute(sql)
results = cur.fetchall()

In [ ]:
import pandas as pd

df = pd.DataFrame(results)
df

,0,1,2,3,4,5,6,7,8,9,10
0,104,2020-01-05,롯데백화점,10014,부츠1,shoes,86873,7,26931,608111,419594
1,103,2020-01-05,이마트,10015,슬리퍼1,shoes,67172,10,14778,671720,523940
2,102,2020-01-06,abc마트,10009,맨투맨1,top,81970,9,27870,737730,486900
3,106,2020-01-08,무신사,10015,슬리퍼1,shoes,67172,9,14778,604548,471546
4,108,2020-01-13,이마트,10019,면바지2,bottom,57460,9,18387,517140,351657
...,...,...,...,...,...,...,...,...,...,...,...
130,230,2020-12-17,현대백화점,10028,구두2,shoes,44360,10,12421,443600,319390
131,235,2020-12-19,현대백화점,10019,면바지2,bottom,57460,9,18387,517140,351657
132,227,2020-12-21,abc마트,10017,흑청바지2,bottom,74516,9,17884,670644,509688
133,232,2020-12-25,abc마트,10014,부츠1,shoes,86873,9,26931,781857,539478


In [ ]:
del df[0]
del df[3]

In [ ]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-05,롯데백화점,부츠1,shoes,86873,7,26931,608111,419594
1,2020-01-05,이마트,슬리퍼1,shoes,67172,10,14778,671720,523940
2,2020-01-06,abc마트,맨투맨1,top,81970,9,27870,737730,486900
3,2020-01-08,무신사,슬리퍼1,shoes,67172,9,14778,604548,471546
4,2020-01-13,이마트,면바지2,bottom,57460,9,18387,517140,351657
...,...,...,...,...,...,...,...,...,...
130,2020-12-17,현대백화점,구두2,shoes,44360,10,12421,443600,319390
131,2020-12-19,현대백화점,면바지2,bottom,57460,9,18387,517140,351657
132,2020-12-21,abc마트,흑청바지2,bottom,74516,9,17884,670644,509688
133,2020-12-25,abc마트,부츠1,shoes,86873,9,26931,781857,539478


In [ ]:
df.to_csv('sales_list.csv', index=False)

## 마무리

In [10]:
cur.close()
conn.close()